In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, select, Float


In [2]:
# Configurando Dados do driver do navegador utilizado para raspagem de dados.
# Navegador utilizado será o Google Chrome.

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(3)
driver.get('https://www.fundsexplorer.com.br/ranking')

In [3]:
# Criando DataFrame Para Armazenar Dados Da Raspagem De Dados.

df = pd.DataFrame(
    columns=[
        'Código',
        'Setor',
        'Preço Atual',
        'Liquidez Diaria',
        'Dívidendo',
        'Dividend Yield',
        'Dividend Yield Anual',
        'Variação Preço',
        'P/VP',
        'Vacância Física',
        'Vacância Financeira'
    ]
)

In [4]:
# Varrendo Os Dados Coletados Na Raspagem De Dados.
# Formatando e Salvando os Dados Coletados.

data = []

for i in range(1, 321):
    code = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[1]/a'.format(i)).text
    sector = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[2]'.format(i)).text
    currentPrice = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[3]'.format(i)).text
    dailyLiquidity = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[4]'.format(i)).text
    dividend = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[5]'.format(i)).text
    dividendYield = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[6]'.format(i)).text
    dividendYieldAnnual = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[13]'.format(i)).text
    priceVariation = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[14]'.format(i)).text
    pvp = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[19]'.format(i)).text
    physicalVacancy = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[24]'.format(i)).text
    financialVacancy = driver.find_element(By.XPATH, '//*[@id="table-ranking"]/tbody/tr[{}]/td[25]'.format(i)).text

    register = {
        'code': code, 
        'sector': sector, 
        'currentPrice': currentPrice, 
        'dailyLiquidity': dailyLiquidity,
        'dividend': dividend, 
        'dividendYield': dividendYield, 
        'dividendYieldAnnual': dividendYieldAnnual,
        'priceVariation': priceVariation,
        'pvp': pvp, 
        'physicalVacancy': physicalVacancy,
        'financialVacancy': financialVacancy
    }
    
    data.append(register)

df = pd.DataFrame.from_dict(data)

### Fazendo o tratamento dos dados

Os seguintes processos foram feitos a seguir para tratar os dados do scrapping:
- Remover dados duplicados
- Remover colunas que não serão utilizadas
- Converter os dados para o tipo correto
- Preenchimento de dados faltantes

In [5]:
processedData = df.copy()
processedData = df.drop_duplicates(keep=False).copy()

processedData.drop(["physicalVacancy", "financialVacancy"], inplace=True, axis=1)

# Convertendo os dados para o tipo float:
for column in processedData.columns[2:]:
    processedData[column] = processedData[column].apply(
        lambda item: item
        if pd.isna(item) or type(item) == float
        else pd.to_numeric(
            item.replace('R$ ', '').replace('%', '').replace('.', '').replace(',', '.'),
            errors="coerce",
        )
    )

# Trantando as colunas categóricas e quantitativas com dados faltantes:
for column in processedData.columns[:2]:
    processedData[column].fillna("Não Disponivel", inplace=True)

for column in processedData.columns[2:]:
    processedData[column].fillna(processedData[column].median(), inplace=True)

# Tratando setores em brancos:
processedData['sector'] = processedData['sector'].apply(
    lambda item: 'Não Disponivel' if item == '' else item
)

#### Tratando outliers

Substituição dos valores discrepantes pelo valor da mediana do atributo.

In [6]:
for column in processedData.columns[2:]:
    median = processedData[column].median()

    q1 = processedData[column].quantile(0.25)
    q3 = processedData[column].quantile(0.75)
    iqr = q3 - q1

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    processedData[column] = processedData[column].apply(
        lambda item: 0.0 if item < lower else median if item > upper else item
    )

### Salvando os dados no banco de dados

In [8]:
engine = create_engine('sqlite:///../database.db', echo=False)
metadata = MetaData()
conn = engine.connect()

In [9]:
fundsTable = Table(
    'funds',
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('code', String(10)),
    Column('sector', String(50)),
    Column('currentPrice', Float),
    Column('dailyLiquidity', Float),
    Column('dividend', Float),
    Column('dividendYield', Float),
    Column('dividendYieldAnnual', Float),
    Column('priceVariation', Float),
    Column('pvp', Float),
)

metadata.create_all(engine)

In [10]:
processedData['id'] = processedData.reset_index().index + 1

# with engine.begin() as connection:
processedData.to_sql('funds', conn, if_exists='replace', index=False)


312

In [11]:
conn.execute(fundsTable.select().where(fundsTable.c.sector == 'Não Disponivel')).fetchall()

[(1, 'AAZQ11', 'Não Disponivel', 9.62, 28390.0, 0.14, 1.46, 5.94, 1.05, 1.0),
 (4, 'AGRX11', 'Não Disponivel', 10.8, 116260.0, 0.18, 1.66, 6.38, 0.37, 1.05),
 (10, 'ARCT11', 'Não Disponivel', 82.7, 28390.0, 0.76, 0.95, 3.88, -0.42, 0.87),
 (27, 'BIME11', 'Não Disponivel', 7.11, 166360.0, 0.09, 1.3, 4.77, 3.28, 0.8),
 (63, 'CPTR11', 'Não Disponivel', 96.99, 145160.0, 1.35, 1.41, 5.82, 0.0, 0.99),
 (70, 'CXCI11', 'Não Disponivel', 79.51, 5470.0, 0.84, 1.08, 4.36, 3.11, 0.89),
 (81, 'EGAF11', 'Não Disponivel', 101.28, 16360.0, 1.2, 1.18, 5.36, 2.52, 1.03),
 (84, 'ERPA11', 'Não Disponivel', 119.0, 9420.0, 0.72, 0.59, 2.28, -0.03, 0.87),
 (92, 'FGAA11', 'Não Disponivel', 9.64, 28390.0, 0.13, 1.34, 5.5, -1.02, 1.01),
 (112, 'GTLG11', 'Não Disponivel', 97.09, 180.0, 0.7, 0.71, 2.89, 3.16, 1.05),
 (121, 'HGAG11', 'Não Disponivel', 34.0, 1330.0, 0.46, 1.25, 6.06, 8.0, 0.885),
 (175, 'LSAG11', 'Não Disponivel', 106.2, 7300.0, 1.75, 1.64, 5.73, 1.41, 1.06),
 (194, 'NCRA11', 'Não Disponivel', 10.1